# Introduction to using ParticleSpy

This is an example notebook for performing particle analysis using ParticleSpy.

This notebook takes a HAADF image of nanoparticles, allows the user to determine the best segmentation method and then performs analysis, producing a plot of particle areas.

Below, we load an image using Hyperspy.

In [ ]:
import hyperspy.api as hs
import particlespy.api as ps
import pkg_resources

filename = pkg_resources.resource_filename('particlespy', 'data/SiO2 HAADF Image.hspy')
haadf = hs.load(filename)

%matplotlib inline
haadf.plot()

Next, we can examine the image and determine the optimum segmentation procedure using the segmentation GUI. Experiment with changing the threshold algorithm and the application of watershedding to obtain the optimum labels. Once happy, click "Get Params" to print the current parameters.

In [ ]:
ps.seg_ui(haadf)

Next, we use the parameters saved when last pressing Update in the GUI to construct a parameter dictionary and use this to perform the segmentation.

In [ ]:
params = ps.parameters()
params.load()
params.store['store_im'] = True
particles = ps.particle_analysis(haadf,params)

Let's interrogate the particle data a little bit. From an initial image we can get particle areas and circularities.

In [ ]:
len(particles.list)
print(particles.list[7].properties['area'])
print(particles.list[7].properties['circularity'])

It is then possible to plot a histogram of particle area.

In [ ]:
particles.plot('area')

## ParticleSpy with EDS data

If you have EDS data collected simultaneously with an image, it is possible to extract maps, spectra and the composition from each segmented particle. Firstly, load the EDS spectrum image.

In [ ]:
eds_filename = pkg_resources.resource_filename('particlespy', 'data/SiO2 EDS Spectrum Image.hspy')
eds = hs.load(eds_filename)

Next, put the HAADF image and EDS spectrum image together in a list to make one acquisition.

In [ ]:
ac = [haadf.isig[:64,:64],eds.inav[:64,:64]]

At this point, we need to set the parameters for the EDS analysis. This can be done by taking our previously used parameters object adding parameters with generate_eds().

In [ ]:
params.generate_eds(elements=['O','Si'], store_maps=True, factors=[1.3,1.0], eds_method='CL')

Now, we need to run ParticleAnalysis again in order to get the accompanying EDS data.

In [ ]:
particles2 = ps.particle_analysis(ac,params)

The extracted maps of each element can be accessed from the particle list from maps['element'].

In [ ]:
particles2.list[2].maps['Si'].as_signal2D([0,1]).plot()

The composition of each particle can be accessed from the particle list using list[particle].composition.

In [ ]:
particles2.list[2].composition

We can plot a radial profile of the particle intensity from both the image and our elemental maps.

In [ ]:
rp = ps.radial_profile(particles.list[2],['Image'],plot=True)

In [ ]:
rp2 = ps.radial_profile(particles2.list[2],['O','Si'],plot=True)